In [21]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.preprocessing import StandardScaler

In [13]:
path = "dataset/images/train"
all_files = glob.glob(path + "/*.csv")
df_list = [pd.read_csv(file) for file in all_files]
train_df = pd.concat(df_list, ignore_index=True)

path = "dataset/images/validation"
all_files = glob.glob(path + "/*.csv")
df_list = [pd.read_csv(file) for file in all_files]
valid_df = pd.concat(df_list, ignore_index=True)

path = "dataset/images/test"
all_files = glob.glob(path + "/*.csv")
df_list = [pd.read_csv(file) for file in all_files]
test_df = pd.concat(df_list, ignore_index=True)

print('Total Train : '+str(len(train_df)))
print('Total Validation : '+str(len(valid_df)))
print('Total Test : '+str(len(test_df)))

Total Train : 25343
Total Validation : 6649
Total Test : 1792


In [22]:
X_train = train_df.iloc[:, 2:].values
y_train = train_df.iloc[:, 1].values

X_valid = valid_df.iloc[:, 2:].values
y_valid = valid_df.iloc[:, 1].values

X_test = test_df.iloc[:, 2:].values
y_test = test_df.iloc[:, 1].values

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_valid = label_encoder.fit_transform(y_valid)
y_test = label_encoder.fit_transform(y_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [24]:
class DataGenerator(Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.X))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = self.X[indices]
        y_batch = self.y[indices]
        return X_batch, y_batch

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

train_generator = DataGenerator(X_train, y_train)
val_generator = DataGenerator(X_valid, y_valid, shuffle=False)
test_generator = DataGenerator(X_test, y_test, shuffle=False)

In [25]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [30]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [31]:
history = model.fit(train_generator, validation_data=val_generator, epochs=100, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
791/791 [==============================] - 5s 5ms/step - loss: 1.2445 - accuracy: 0.5187 - val_loss: 1.2234 - val_accuracy: 0.5234 - lr: 0.0010
Epoch 2/100
791/791 [==============================] - 4s 5ms/step - loss: 1.2464 - accuracy: 0.5188 - val_loss: 1.2177 - val_accuracy: 0.5272 - lr: 0.0010
Epoch 3/100
791/791 [==============================] - 4s 6ms/step - loss: 1.2464 - accuracy: 0.5226 - val_loss: 1.2179 - val_accuracy: 0.5273 - lr: 0.0010
Epoch 4/100
791/791 [==============================] - 4s 5ms/step - loss: 1.2468 - accuracy: 0.5173 - val_loss: 1.2154 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 5/100
791/791 [==============================] - 4s 5ms/step - loss: 1.2447 - accuracy: 0.5185 - val_loss: 1.2130 - val_accuracy: 0.5300 - lr: 0.0010
Epoch 6/100
791/791 [==============================] - 5s 6ms/step - loss: 1.2421 - accuracy: 0.5200 - val_loss: 1.2141 - val_accuracy: 0.5279 - lr: 0.0010
Epoch 7/100
791/791 [==============================] - 4s 5ms/st

In [32]:
model.evaluate(test_generator, verbose=1)

56/56 [==============================] - 0s 2ms/step - loss: 1.3447 - accuracy: 0.4933


[1.3446942567825317, 0.4933035671710968]

---